From [Learn](https://learn.microsoft.com/en-us/training/modules/orchestrate-semantic-kernel-multi-agent-solution/3-understand-agent-orchestration):

Workflows in the Microsoft Agent Framework allow developers to design, monitor, and control how multiple AI agents and logic components interact to complete complex tasks. They bring structure, flexibility, and transparency to agent-driven applications.

One of the ways to bring in control and manage agent workflows is to add human approvals. When agents require any user input, for example to approve a function call, this is referred to as a human-in-the-loop pattern. 

An agent run that requires user input will complete with a response that indicates what input is required from the user instead of completing with a final answer. The caller of the agent is then responsible for getting the required input from the user and passing it back to the agent as part of a new agent run.


### Example 1 - Function Tools with Human-In-The-Loop approvals



In [7]:
import os
from dotenv import load_dotenv
from agent_framework.azure import AzureOpenAIChatClient

load_dotenv()
endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
api_key = os.getenv("AZURE_OPENAI_API_KEY")
api_version = os.getenv("AZURE_OPENAI_API_VERSION")
deployment = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")

chat_client=AzureOpenAIChatClient(
        endpoint=endpoint,
        api_key=api_key,
        api_version=api_version,
        deployment_name=deployment,
    )

In [34]:
from typing import Annotated
from agent_framework import ai_function, ChatMessage, Role
from agent_framework.azure import AzureOpenAIResponsesClient


@ai_function
def get_weather(location: Annotated[str, "The city and state, e.g. San Francisco, CA"]) -> str:
    """Get the current weather for a given location."""
    return f"The weather in {location} is sunny with a high of 30°C."

@ai_function(approval_mode="always_require")
def get_weather_detail(location: Annotated[str, "The city and state, e.g. San Francisco, CA"]) -> str:
    """Get detailed weather information for a given location."""
    return f"The weather in {location} is cloudy with a high of 15°C, humidity 88%."

agent = chat_client.create_agent(
    name="WeatherAgent",
    instructions="You are a helpful weather assistant.",
    tools=[get_weather_detail, get_weather],
)


result = await agent.run("What is the detailed weather like in Amsterdam?")

if result.user_input_requests:
    for user_input_needed in result.user_input_requests:
        print(f"Function: {user_input_needed.function_call.name}")
        print(f"Arguments: {user_input_needed.function_call.arguments}")

        
    # Get user approval (in a real application, this would be interactive)
    user_approval = False  # or False to reject

    # Create the approval response
    approval_message = ChatMessage(
        role=Role.USER, 
        contents=[user_input_needed.create_response(user_approval)]
    )

    final_result = await agent.run([
        "What is the detailed weather like in Amsterdam?",
        ChatMessage(role=Role.ASSISTANT, contents=[user_input_needed]),
        approval_message
    ])

print(final_result.text)

Function: get_weather_detail
Arguments: {"location":"Amsterdam, Netherlands"}
The current weather in Amsterdam, Netherlands is sunny, with a high temperature of 30°C. If you need more detailed weather information or forecasts, please let me know!


requests and responses seem super important:
https://learn.microsoft.com/en-us/agent-framework/tutorials/workflows/requests-and-responses?pivots=programming-language-python


https://github.com/microsoft/agent-framework/blob/main/python/samples/getting_started/workflows/agents/workflow_as_agent_human_in_the_loop.py

love this one - https://github.com/microsoft/agent-framework/blob/main/python/samples/getting_started/workflows/human-in-the-loop/guessing_game_with_human_input.py
